In [1]:
# 安裝套件 for https://pypi.org/project/interpret-community/
# This package has been tested with Python 3.5, 3.6 and 3.7
# !pip install interpret-community

In [2]:
import pandas as pd
df = pd.read_csv('diabetes.csv')

In [8]:
# 取出 Ｘ特徵, y標籤
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = df.iloc[:, 1:-1].values , df.iloc[:, -1].values
X[:2]

array([[0.00000000e+00, 1.71000000e+02, 8.00000000e+01, 3.40000000e+01,
        2.30000000e+01, 4.35097259e+01, 1.21319135e+00, 2.10000000e+01],
       [8.00000000e+00, 9.20000000e+01, 9.30000000e+01, 4.70000000e+01,
        3.60000000e+01, 2.12405757e+01, 1.58364981e-01, 2.30000000e+01]])

In [4]:
# 資料處理
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
X_ss = ss.fit_transform(X)

In [5]:
# 區分訓練與驗證數據 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

c:\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7883333333333333

In [10]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
tab_explainer = TabularExplainer(model,
                                 X_train, 
                                 features=features, 
                                 classes=labels)
print(tab_explainer, "ready!")

Could not import lightgbm, required if using LGBMExplainableModel


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


TabularExplainer ready!


The option feature_dependence has been renamed to feature_perturbation!
The option feature_perturbation="independent" is has been renamed to feature_perturbation="interventional"!


In [11]:
# you can use the training data or the test data here
global_tab_explanation = tab_explainer.explain_global(X_train)

# Get the top features by importance
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

Pregnancies : 0.7738326549805316
Age : 0.5674265497913503
BMI : 0.40880737624654634
SerumInsulin : 0.3802614327979297
DiabetesPedigree : 0.2836407976562152
TricepsThickness : 0.28358333438249866
PlasmaGlucose : 0.2197000888016443
DiastolicBloodPressure : 0.15585744638475404


In [12]:
# local
# Get the observations we want to explain (the first two)
X_explain = X_test[0:2]

# Get predictions
predictions = model.predict(X_explain)

# Get local explanations
local_tab_explanation = tab_explainer.explain_local(X_explain)

# Get feature names and importance for each possible label
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

Support for not-diabetic
	Observation 1
		 Age : 0.3585110577225474
		 SerumInsulin : 0.3343109165835533
		 TricepsThickness : 0.2909764596855784
		 DiabetesPedigree : 0.18145223270795766
		 PlasmaGlucose : 0.12522926683692426
		 DiastolicBloodPressure : -0.15789666299543417
		 BMI : -0.6518170768933169
		 Pregnancies : -0.9910960532871952
		 ----------
		 Total: -0.5103298596393854 Prediction: not-diabetic
	Observation 2
		 Pregnancies : 0.5931438027765453
		 TricepsThickness : 0.35884093063340394
		 SerumInsulin : 0.30069415456832693
		 DiabetesPedigree : 0.2915524692076788
		 DiastolicBloodPressure : 0.09920877570353023
		 BMI : 0.0879021829688315
		 PlasmaGlucose : 0.08343416621050334
		 Age : -0.2200669769844407
		 ----------
		 Total: 1.5947095050843794 Prediction: not-diabetic
Support for diabetic
	Observation 1
		 Pregnancies : 0.9910960532871952
		 BMI : 0.6518170768933169
		 DiastolicBloodPressure : 0.15789666299543417
		 PlasmaGlucose : -0.12522926683692426
		 DiabetesPedigr